In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
class AdvDetector(nn.Module):
    def __init__(self, in_channels, pooling1=False, pooling2=False):
        super(AdvDetector, self).__init__()
        
        self.pooling1=pooling1
        self.pooling2=pooling2
        
        self.conv1=nn.Conv2d(in_channels=in_channels, out_channels=96,kernel_size=3,stride=1, padding=1)
        self.layer2=nn.BatchNorm2d(96)
        self.layer3=nn.ReLU()
        self.layer4=nn.MaxPool2d(2,2) #1st pooling layer
        self.layer5=nn.Conv2d(in_channels=96, out_channels=192,kernel_size=3,stride=1,padding=1)
        self.layer6=nn.BatchNorm2d(192)
        self.layer7=nn.ReLU()
        self.layer8=nn.MaxPool2d(2,2)  #2nd pooling layer
        self.layer9=nn.Conv2d(in_channels=192, out_channels=192,kernel_size=3,stride=1, padding=1)
        self.layer10=nn.BatchNorm2d(192)
        self.layer11=nn.ReLU()
        self.layer12=nn.Conv2d(in_channels=192,out_channels=2,kernel_size=1,stride=1)
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
    def forward(self,x):        
        x = self.conv1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        if self.pooling1==True:
            x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        if self.pooling2==True:
            x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        
        x = self.adaptive_avg_pool(x)
        return x

In [5]:
net = AdvDetector(64,False,False)
print(net)
net.train()

AdvDetector(
  (conv1): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer3): ReLU()
  (layer4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer5): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer7): ReLU()
  (layer8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer10): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer11): ReLU()
  (layer12): Conv2d(192, 2, kernel_size=(1, 1), stride=(1, 1))
  (adaptive_avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
)


AdvDetector(
  (conv1): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer3): ReLU()
  (layer4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer5): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer7): ReLU()
  (layer8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer10): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer11): ReLU()
  (layer12): Conv2d(192, 2, kernel_size=(1, 1), stride=(1, 1))
  (adaptive_avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
)

In [6]:
temp_data = np.random.randn(128, 64, 8, 8)
temp_data = torch.Tensor(temp_data)
out = net(temp_data)
out

tensor([[[[-4.4611e-02]],

         [[-2.0751e-01]]],


        [[[ 1.7841e-02]],

         [[-2.2638e-01]]],


        [[[-3.1031e-02]],

         [[-1.6234e-01]]],


        [[[ 3.8574e-02]],

         [[-1.5390e-01]]],


        [[[ 3.8549e-02]],

         [[-1.4190e-01]]],


        [[[ 1.1267e-02]],

         [[-2.2500e-01]]],


        [[[-5.4820e-02]],

         [[-1.7719e-01]]],


        [[[ 6.8073e-02]],

         [[-1.1434e-01]]],


        [[[-5.7589e-02]],

         [[-2.1635e-01]]],


        [[[ 4.9847e-02]],

         [[-1.2086e-01]]],


        [[[-3.4997e-02]],

         [[-1.7323e-01]]],


        [[[ 4.2124e-02]],

         [[-1.9158e-01]]],


        [[[-3.7102e-02]],

         [[-1.7714e-01]]],


        [[[-1.0317e-02]],

         [[-1.6422e-01]]],


        [[[ 5.3336e-02]],

         [[-1.4293e-01]]],


        [[[ 2.8918e-02]],

         [[-1.1982e-01]]],


        [[[ 9.5351e-03]],

         [[-1.9380e-01]]],


        [[[-2.5978e-02]],

         [[-1.3737e-0

In [7]:
print(out.shape)
out[1]

torch.Size([128, 2, 1, 1])


tensor([[[ 0.0178]],

        [[-0.2264]]], grad_fn=<SelectBackward>)

In [8]:
target = torch.ones((128))
out = out.reshape(128, 2)

print(target.shape, out.shape)

torch.Size([128]) torch.Size([128, 2])


In [9]:
criterion = nn.CrossEntropyLoss()
out = out.float()

loss = criterion(out, target.long())

# loss.backward()
print(loss)

tensor(0.7708, grad_fn=<NllLossBackward>)


In [ ]:
file=np.load('c1s_train.npy')

In [ ]:
print(file)
print(file.shape)

[[[[0.02511253 0.         0.         ... 0.14928018 0.14467147
    0.15240252]
   [0.         0.         0.         ... 0.08861911 0.07271574
    0.10648758]
   [0.         0.         0.         ... 0.0020269  0.
    0.05558911]
   ...
   [0.17123878 0.12336614 0.09099106 ... 0.         0.
    0.        ]
   [0.16745529 0.13568199 0.11067381 ... 0.01278207 0.
    0.        ]
   [0.17070366 0.15572256 0.14748897 ... 0.15146844 0.04827313
    0.05673229]]

  [[0.00252416 0.12250652 0.19009921 ... 0.22631727 0.21007049
    0.20509145]
   [0.         0.06003428 0.27195188 ... 0.53127867 0.51131105
    0.46020424]
   [0.         0.10832304 0.38874257 ... 0.5309818  0.56250304
    0.5295518 ]
   ...
   [0.83916825 0.70556504 0.69942427 ... 0.31367826 0.
    0.46911907]
   [0.80559593 0.8920938  0.9936337  ... 0.5336421  0.06349538
    0.49580386]
   [0.70889914 0.8461572  1.0714632  ... 0.7713433  0.24859405
    0.45015213]]

  [[0.         0.         0.         ... 0.         0.
    0.     

In [31]:
#device = torch.device("cuda" if torch.cuda.is_available() 
                                 # else "cpu")
device="cpu"

In [32]:
epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

In [34]:
X_train=temp_data 
y_train=target

In [35]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

#test_data = testData(torch.FloatTensor(X_test))

In [36]:
train_loader = DataLoader(dataset=train_data, batch_size=4, shuffle=True)
#test_loader = DataLoader(dataset=test_data, batch_size=1)

In [37]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()


In [39]:
# train
EPOCHS=20
net.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = net(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = accuracy(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

RuntimeError: ignored

In [ ]:
y_pred_list = []
net.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = net(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]